In [1]:
print("ok")

ok


In [2]:
# Install some required packages

!pip install pypdf2
!pip install google-cloud-aiplatform
!pip install google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.4 MB/s eta 0:00:00


In [3]:
from google.cloud import storage
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform
import PyPDF2

import re
import os
import random
import json
import uuid


In [4]:
%ls

stats.pdf


In [5]:
# Initialize some variable

# project="your_GCP_project_id"
location="us-central1"

pdf_path="stats.pdf"
bucket_name = "stats-content"
embed_file_path = "stats_embeddings.json"
sentence_file_path = "stats_sentences.json"
index_name="stats_index"



In [6]:
# helper

def extract_sentences_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            if page.extract_text() is not None:
                text += page.extract_text() + " "
    sentences = [sentence.strip() for sentence in text.split('. ') if sentence.strip()]
    return sentences

In [7]:
def generate_text_embeddings(sentences) -> list:
  # aiplatform.init(project=project,location=location)
  model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
  embeddings = model.get_embeddings(sentences)
  vectors = [embedding.values for embedding in embeddings]
  return vectors

In [8]:
def generate_and_save_embeddings(pdf_path, sentence_file_path, embed_file_path):
    def clean_text(text):
        cleaned_text = re.sub(r'\u2022', '', text)  # Remove bullet points
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Remove extra whitespaces and strip
        return cleaned_text

    sentences = extract_sentences_from_pdf(pdf_path)
    if sentences:
        embeddings = generate_text_embeddings(sentences)

        with open(embed_file_path, 'w') as embed_file, open(sentence_file_path, 'w') as sentence_file:
            for sentence, embedding in zip(sentences, embeddings):
                cleaned_sentence = clean_text(sentence)
                id = str(uuid.uuid4())

                embed_item = {"id": id, "embedding": embedding}
                sentence_item = {"id": id, "sentence": cleaned_sentence}

                json.dump(sentence_item, sentence_file)
                sentence_file.write('\n')
                json.dump(embed_item, embed_file)
                embed_file.write('\n')

In [9]:
def upload_file(bucket_name,file_path):
    storage_client = storage.Client()
    bucket = storage_client.create_bucket(bucket_name,location=location)
    blob = bucket.blob(file_path)
    blob.upload_from_filename(file_path)

In [10]:
def create_vector_index(bucket_name, index_name):
    lakeside_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name = index_name,
    contents_delta_uri = "gs://"+bucket_name,
    dimensions = 768,
    approximate_neighbors_count = 10,
    )

    lakeside_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name = index_name,
    public_endpoint_enabled = True
    )

    lakeside_index_endpoint.deploy_index(
    index = lakeside_index, deployed_index_id = index_name
    )

In [11]:
generate_and_save_embeddings(pdf_path,sentence_file_path,embed_file_path)
upload_file(bucket_name,sentence_file_path)

In [12]:
create_vector_index(bucket_name, index_name)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/1060807096482/locations/us-central1/indexes/3439259177543794688/operations/7478939149387431936
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex created. Resource name: projects/1060807096482/locations/us-central1/indexes/3439259177543794688
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:To use this MatchingEngineIndex in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:index = aiplatform.MatchingEngineIndex('projects/1060807096482/locations/us-central1/indexes/3439259177543794688')
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Creating MatchingEngineIndexEndpoint
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Create MatchingEngi

ResourceExhausted: 429 The following quotas are exceeded: MatchingEngineDeployedIndexNodes 8: The following quotas are exceeded: MatchingEngineDeployedIndexNodes

In [13]:
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import json
import os

In [ ]:
# project=”YOUR_GCP_PROJECT”
location="us-central1"
sentence_file_path = "stats_sentences.json"
index_name="stats_index" #Get this from the console or the previous step

In [15]:
# aiplatform.init(project=project,location=location)
# vertexai.init()
model = GenerativeModel("gemini-pro")
lakeside_index_ep = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name="5030296483400777728")

In [16]:
def generate_text_embeddings(sentences) -> list:
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
    embeddings = model.get_embeddings(sentences)
    vectors = [embedding.values for embedding in embeddings]
    return vectors


def generate_context(ids,data):
    concatenated_names = ''
    for id in ids:
        for entry in data:
            if entry['id'] == id:
                concatenated_names += entry['sentence'] + "\n"
    return concatenated_names.strip()



def load_file(sentence_file_path):
  data = []
  with open(sentence_file_path,'r') as f:
    for line in f:
      entry = json.loads(line)
      data.append(entry)
  return data





In [17]:
data=load_file(sentence_file_path)
data

[{'id': '9f6055cb-fd9f-4fb4-a8e6-f8570c267098',
  'sentence': 'Importance and the use of correlation in Statistics Introduction Correlation is a statistical measure that expresses the extent to which two variables are linearly related. It is a common tool for describing simple relationships without making a statement about cause and effect. Correlation coefficients range from -1 to 1, with a value of 0 indicating no linear relationship between the two variables, a value of 1 indicating a perfect positive linear relationship, and a value of -1 indicating a perfect negative linear relationship. Correlation is important in statistics because it can be used to 1. Identify relationships between variables: Correlation can be used to identify whether there is a relationship between two variables, and if so, whether the relationship is positive or negative. This information can be useful for understanding the relationships between different factors in a complex system. 2. Make predictions: If 

In [18]:
query=["what is correlation?"]

qry_emb=generate_text_embeddings(query)

In [19]:
qry_emb

[[0.0031879430171102285,
  0.030173784121870995,
  -0.046163856983184814,
  -0.010091220960021019,
  -0.026898862794041634,
  -0.05871526896953583,
  0.022807922214269638,
  0.02700706385076046,
  -0.021587025374174118,
  0.03865209221839905,
  -0.014295038767158985,
  -0.04151195287704468,
  0.035884130746126175,
  -5.352220614440739e-05,
  -0.039067499339580536,
  0.007143754046410322,
  -0.02639135904610157,
  -0.015075067058205605,
  0.043786194175481796,
  -0.019433336332440376,
  -0.03991476446390152,
  -0.060704465955495834,
  -0.019392838701605797,
  -0.01959632895886898,
  -0.019830776378512383,
  -0.10087926685810089,
  0.00258005503565073,
  0.007023989222943783,
  -0.025977298617362976,
  -0.028022313490509987,
  0.03841837868094444,
  0.01058691181242466,
  0.017269540578126907,
  0.003389855148270726,
  -0.024171901866793633,
  0.040811534970998764,
  0.016141369938850403,
  0.026479193940758705,
  -0.03963363170623779,
  -0.013527167029678822,
  0.03784491494297981,
  -0

In [20]:
response = lakeside_index_ep.find_neighbors(
    deployed_index_id = index_name,
    queries = [qry_emb[0]],
    num_neighbors = 10
)

ServiceUnavailable: 503 DNS resolution failed for 1645167058.us-central1-1060807096482.vdb.vertexai.goog:443: C-ares status is not ARES_SUCCESS qtype=A name=1645167058.us-central1-1060807096482.vdb.vertexai.goog is_balancer=0: Domain name not found

In [ ]:


matching_ids = [neighbor.id for sublist in response for neighbor in sublist]

context = generate_context(matching_ids,data)
prompt=f"Based on the context delimited in backticks, answer the query. ```{context}``` {query}"

chat = model.start_chat(history=[])
response = chat.send_message(prompt)
print(response.text)

In [1]:
from llama_index.core import PromptTemplate

In [2]:
faq_prompt_str = (
    "## Very Important Instructions"
    "- Respond as if you were conversing with a person\n"
    "- Only answer with information provided in the context\n"
    "- Do NOT give 'additional help' OR 'additional information' OR 'notes' OR 'additional guidance\n"
    "- After providing answer, do NOT ask user to contact additional support\n"
    "- Do not answer unrelated questions about companies that are NOT MoneyGram like Western Union, Microsoft, etc.\n"
    "- Do not include information about unrelated companies that are NOT MoneyGram like Western Union, Microsoft, etc. in your answers\n"
)

faq_qa_tmpl_str = (
    "You are a friendly MoneyGram chatbot that chats with a user and answers their question concisely "
    "using the below context. "
    "If you cannot find an answer in the context, you "
    "should respond with the following response 'Hmm, I'm sorry I can't help you with that'. "
    "Format answers to maximize readability by ensuring each element of a list is separated by a newline. "
    "Break up long paragraphs of text into smaller easier to read chunks. "
    "\n"
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "{prompt_str}\n"
    "---------------------\n"
    "Query: {query_str}\n"
    "Answer: "
)

final_formatting = (
    """
    Format the below text in to more readable format by breaking up long paragraphs of text into smaller easier to read chunks.
    DO NOT add or generate any new text.

    Text:

    {chat_transcript_str}

    """
)




In [3]:
faq_qa_tmpl = PromptTemplate(faq_qa_tmpl_str.replace('{prompt_str}', faq_prompt_str))

In [4]:
faq_qa_tmpl

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template="You are a friendly MoneyGram chatbot that chats with a user and answers their question concisely using the below context. If you cannot find an answer in the context, you should respond with the following response 'Hmm, I'm sorry I can't help you with that'. Format answers to maximize readability by ensuring each element of a list is separated by a newline. Break up long paragraphs of text into smaller easier to read chunks. \nContext information is below.\n---------------------\n{context_str}\n---------------------\n## Very Important Instructions- Respond as if you were conversing with a person\n- Only answer with information provided in the context\n- Do NOT give 'additional help' OR 'additional information' OR 'notes' OR 'additional guidance\n- After providing answer, do NOT as

In [5]:
import pandas as pd
import numpy as np
import json
import os

In [17]:
def mul(i):

    return i ** i

# Using the map function

x = map(mul, (3, 5, 7, 11, 13))

print (x)

print(list(x))

[27, 3125, 823543, 285311670611, 302875106592253]


In [32]:
my_list = [2,3,4,5,6,7,8,9]

updated_list = list(map(lambda x: x * 10, my_list))

In [33]:
print((updated_list))

[20, 30, 40, 50, 60, 70, 80, 90]


In [38]:
single_dict = {"name": ["A","B","C","d","B"], "age":[12,23,56,44, 30], "city": ["New York","New York","New York","New York","New York"]}



In [39]:
DF = pd.DataFrame(single_dict)
DF

,name,age,city
0,A,12,New York
1,B,23,New York
2,C,56,New York
3,d,44,New York
4,B,30,New York


In [45]:
DF.to_csv('C:\PycharmProjects\RAG_Projects\P1\csv\\output4.csv', index=False)

In [46]:
import pandas as pd
import glob

df = pd.concat([pd.read_csv(f) for f in glob.glob('C:\PycharmProjects\RAG_Projects\P1\csv\*.csv')], ignore_index=True)


In [47]:
df

,name,age,city
0,A,12,New York
1,B,23,New York
2,C,56,New York
3,d,44,New York
4,B,30,New York
5,A,12,New York
6,B,23,New York
7,C,56,New York
8,d,44,New York
9,B,30,New York


In [94]:
df.sum()

name                            ABCdBABCdBABCdBABCdBABCdB
age                                                   825
city    New YorkNew YorkNew YorkNew YorkNew YorkNew Yo...
dtype: object

In [96]:
df.age.mean()


33.0

In [97]:
df.age.median()


30.0

In [98]:
df.age.min()


12

In [99]:
df.age.max()


56

In [100]:
df.age.count()


25

In [101]:
df.age.var()


250.0

In [102]:
df.age.std()


15.811388300841896

In [104]:
# df.age.corr()

In [54]:
your_list = [10, 20, 4, 45, 99, 20, 99]

In [57]:
second_highest = sorted(set(your_list))[0]
second_highest

4

In [59]:
your_list = [10, 20, 4, 45, 99, 20, 99]

highest = second_highest = float('-inf')  # Initialize with negative infinity
print(highest,second_highest)

for num in your_list:
    if num > highest:
        second_highest = highest  # Update second-highest
        highest = num  # Update highest
        print(highest,second_highest)
    elif num > second_highest and num != highest:
        second_highest = num  # Update second-highest if it's not equal to the highest
        print(highest,second_highest)
print(f"second_highest number is {second_highest} and highest number is {highest}")  # Output: 45



-inf -inf
10 -inf
20 10
45 20
99 45
second_highest number is 45 and highest number is 99


In [60]:
n = len(your_list)
print(n)
for i in range(n):
    for j in range(n - 1): 
        print(your_list[j],your_list[j+1]) # Compare each pair of adjacent elements
        if your_list[j] > your_list[j + 1]:  # Swap if needed
            your_list[j], your_list[j + 1] = your_list[j + 1], your_list[j]
print(your_list)

7
[4, 10, 20, 20, 45, 99, 99]


In [61]:
def bubblesort(list1):  
    n=len(list1)
    for i in range(n):
        for j in range(n-1):
            print(list1[j],list1[j+1])
            if list1[j]>list1[j+1]:
                print("True")
                list1[j],list1[j+1]=list1[j+1],list1[j]
    return list1

In [63]:
arr = [64, 34, 25, 12, 22, 11, 90]

In [64]:
result=bubblesort(arr)
result

64 34
True
64 25
True
64 12
True
64 22
True
64 11
True
64 90
34 25
True
34 12
True
34 22
True
34 11
True
34 64
64 90
25 12
True
25 22
True
25 11
True
25 34
34 64
64 90
12 22
22 11
True
22 25
25 34
34 64
64 90
12 11
True
12 22
22 25
25 34
34 64
64 90
11 12
12 22
22 25
25 34
34 64
64 90
11 12
12 22
22 25
25 34
34 64
64 90


[11, 12, 22, 25, 34, 64, 90]

In [65]:
from collections import Counter

text = "embeddings"
word_count = Counter(text)  # Count each character in the string
print(word_count)


Counter({'e': 2, 'd': 2, 'm': 1, 'b': 1, 'i': 1, 'n': 1, 'g': 1, 's': 1})


In [69]:
text = "embeddinembeddingsembeddingsgs"

In [70]:
count={}

In [71]:
for i in text:
    count[i]=count.get(i,0)+1
print(count)


{'e': 6, 'm': 3, 'b': 3, 'd': 6, 'i': 3, 'n': 3, 'g': 3, 's': 3}


In [72]:
import json

# Sample JSON file
json_data = '{"name": "John", "age": 30, "city": "New York"}'

# Parse JSON
data = json.loads(json_data)  # Convert JSON string to Python dictionary
print(data["name"])  # Output: John


John


In [73]:
data = {
    "name": "Alice",
    "age": 25,
    "city": "Los Angeles"
}


In [74]:
# Write JSON to a file
with open("data.json", "w") as file:
    json.dump(data, file, indent=4)

In [75]:
import json

json_data = '{"name": "Mike", "skills": ["Python", "SQL"], "age": 35}'

data = json.loads(json_data)
data


{'name': 'Mike', 'skills': ['Python', 'SQL'], 'age': 35}

In [76]:
print("Name:", data["name"])  # Output: Mike
print("Skills:", ", ".join(data["skills"]))  # Output: Python, SQL

Name: Mike
Skills: Python, SQL


In [80]:
", ".join(data["skills"])

'Python, SQL'

In [81]:
print("Skills:", data["skills"])

Skills: ['Python', 'SQL']


In [83]:
import json

json_data = '''
[
    {"id": 1, "name": "Anna", "score": 85},
    {"id": 2, "name": "Bob", "score": 92},
    {"id": 3, "name": "Cathy", "score": 78}
]
'''

data = json.loads(json_data)
d=[x for x in data if x['score']<80]
print(d)
filtered_data = [student for student in data if student["score"] > 80]

print(json.dumps(filtered_data, indent=4))

[{'id': 3, 'name': 'Cathy', 'score': 78}]
[
    {
        "id": 1,
        "name": "Anna",
        "score": 85
    },
    {
        "id": 2,
        "name": "Bob",
        "score": 92
    }
]


In [84]:
text = "Hello World! This is Text Data Cleaning."
cleaned_text = text.lower()
print(cleaned_text)
# Output: hello world! this is text data cleaning.


hello world! this is text data cleaning.


In [85]:
import string

text = "Hello, World! This is: text-data cleaning?"
cleaned_text = text.translate(str.maketrans("", "", string.punctuation))
print(cleaned_text)
# Output: Hello World This is textdata cleaning


Hello World This is textdata cleaning


In [88]:
import re

text = "This is text data with numbers 12345."
cleaned_text = re.sub(r'\d+', '', text)
print(cleaned_text)
# Output: This is text data with numbers .


This is text data with numbers .


In [89]:
text = "   This is   text   data   cleaning.   "
cleaned_text = " ".join(text.split())
print(cleaned_text)
# Output: This is text data cleaning.


This is text data cleaning.


In [90]:
text.split()

['This', 'is', 'text', 'data', 'cleaning.']

In [1]:
# from nltk.corpus import stopwords
# import nltk
# nltk.download('stopwords')

# text = "This is a sample text for cleaning stopwords."
# stop_words = set(stopwords.words('english'))
# cleaned_text = " ".join(word for word in text.split() if word.lower() not in stop_words)
# print(cleaned_text)
# # Output: sample text cleaning stopwords.


In [2]:
# from nltk.tokenize import word_tokenize
# nltk.download('punkt')

# text = "This is a sample text for tokenization."
# tokens = word_tokenize(text)
# print(tokens)
# # Output: ['This', 'is', 'a', 'sample', 'text', 'for', 'tokenization', '.']


In [3]:
my_list = [1, 2, 3]

# Using a for loop
for item in my_list:
    print(item)

# Using enumerate for index and value
for index, value in enumerate(my_list):
    print(f"Index: {index}, Value: {value}")


1
2
3
Index: 0, Value: 1
Index: 1, Value: 2
Index: 2, Value: 3


In [4]:
class MyClass:
    # Class attributes
    class_attribute = "This is a class attribute"

    def __init__(self, attribute1, attribute2):
        # Instance attributes
        self.attribute1 = attribute1
        self.attribute2 = attribute2

    def my_method(self):
        print("This is a method of MyClass")

# Create an object
my_object = MyClass("value1", "value2")

In [5]:
my_object.attribute1

'value1'

In [6]:
my_object.my_method()

This is a method of MyClass
